
# 🧪 Self-Supervised Prompt Optimization (OvO) — Colab Notebook

This Colab walks you through a **self-supervised prompt optimization** loop based on Output-vs-Output (OvO) pairwise judging.

**What you'll get:**
- A single-file runner with **early stop**, **retry/backoff**, **token-cost tracking**, and **judge/executor decoupling**
- Example `tasks.json` and `prompt_seed.txt`
- One-click run to produce `best_prompt.md`, `trace.json`, and `run_report.md`

> Works with **Mistral** or any **OpenAI-compatible** endpoint. Just set your API key in the next cell.


## 1) Setup API key and provider

In [ ]:

import os

PROVIDER = "mistral"  # or "openai"
os.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_API_KEY", "sk-REPLACE_ME")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "")

print("Provider:", PROVIDER)
print("MISTRAL_API_KEY set?", bool(os.environ.get("MISTRAL_API_KEY")))
print("OPENAI_API_KEY set?", bool(os.environ.get("OPENAI_API_KEY")))


## 2) Install minimal dependencies

In [ ]:

!pip -q install requests


## 3) Prepare example tasks and initial prompt

In [ ]:

import json, textwrap, pathlib

example_tasks = [
  {"input": "Rewrite the product description in a friendly tone for parents of 6–10 year-old kids."},
  {"input": "Summarize the following paragraph into 3 bullet points: Paste your text here."},
  {"input": "Classify the intent (support/sales/other): 'I cannot access premium features after payment.'"},
  {"input": "Extract a JSON with keys {company, title, years} from this bio: Paste bio text here."},
  {"input": "Turn these notes into a clear email with subject + body: Paste notes here."}
]

pathlib.Path("tasks.json").write_text(json.dumps(example_tasks, indent=2), encoding="utf-8")

prompt_seed = textwrap.dedent("""
You are a senior writing & information assistant.

GOAL
- Produce accurate, concise, and well-structured outputs that directly satisfy the Task.
- Prefer bullet points and JSON where appropriate.
- Never invent facts; if input is ambiguous, ask 1 brief clarification question.

INPUT FORMAT
- You will receive: ### Task ... ### Answer
- Read Task carefully. If it requests JSON, return valid JSON only.

STYLE & QUALITY
- Clarity > verbosity. Use plain language.
- If transforming text, preserve meaning and key details.
- For summaries: 3–5 bullets, each ≤ 18 words.
- For classifications: return a single lowercase label from the allowed set.

SAFETY & HONESTY
- If essential info is missing, return: "NEED-CLARIFICATION: <one short question>"

Now wait for Task and then produce the Answer.
""")

pathlib.Path("prompt_seed.txt").write_text(prompt_seed, encoding="utf-8")

print("Wrote tasks.json and prompt_seed.txt")


## 4) Create the SPO runner (single-file)

In [ ]:

%%bash
cat > spo_runner.py << 'PYCODE'
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, json, time, random, argparse, pathlib, sys
from dataclasses import dataclass, asdict
from typing import List, Dict, Any, Tuple
import requests

def _chat(url, api_key, model, messages, max_tokens=512, temperature=0.7, timeout=60):
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    payload = {"model": model, "messages": messages, "temperature": temperature, "max_tokens": max_tokens}
    resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
    resp.raise_for_status()
    data = resp.json()
    text = data["choices"][0]["message"]["content"]
    usage = data.get("usage", {})
    return text, usage

PROVIDERS = {
    "mistral": {"base_url": "https://api.mistral.ai/v1/chat/completions", "key_env": "MISTRAL_API_KEY"},
    "openai":  {"base_url": "https://api.openai.com/v1/chat/completions", "key_env": "OPENAI_API_KEY"},
}

SYS_EXEC = "You are a helpful expert that strictly follows the prompt to solve the task."
SYS_JUDGE = "You are an impartial judge. Given a TASK and two CANDIDATE ANSWERS, decide which is BETTER for the task.\nReturn ONLY 'A' or 'B' on the first line. Then give ONE short sentence explaining why."

def judge_prompt(task, ansA, ansB):
    return [
        {"role":"system","content":SYS_JUDGE},
        {"role":"user","content":(
            "TASK:\n"+task+"\n\nCANDIDATE A:\n"+ansA+"\n\nCANDIDATE B:\n"+ansB+"\n\n"
            "Answer with 'A' or 'B' ONLY on the first line, then a brief reason."
        )}
    ]

OPT_SYS = "You are a prompt engineer. Improve the given PROMPT for higher quality, truthful, concise outputs."
def opt_user(best_prompt, brief_feedback):
    return (
        "Here is the current best PROMPT (triple backticks). Improve it while keeping task intent unchanged.\n"
        "Focus on: clarity, structure, constraints, evaluation rubric, step-by-step reasoning hints.\n"
        f"Feedback signals:\n- {brief_feedback}\n\n"
        "```PROMPT\n" + best_prompt + "\n```"
        "\nReturn ONLY the improved prompt."
    )

@dataclass
class Config:
    provider: str
    api_key: str
    base_url: str
    exec_model: str
    judge_model: str
    rounds: int = 10
    k: int = 4
    n: int = 3
    patience: int = 2
    max_tokens_exec: int = 512
    max_tokens_judge: int = 256
    temperature_exec: float = 0.2
    temperature_judge: float = 0.0
    seed: int = 42
    out_dir: str = "runs/colab"

def set_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

def ensure_dir(p): pathlib.Path(p).mkdir(parents=True, exist_ok=True)

def backoff_retry(fn, retries=3, base=1.4, max_wait=10, *args, **kwargs):
    for i in range(retries+1):
        try:
            return fn(*args, **kwargs)
        except Exception:
            if i==retries: raise
            import time; time.sleep(min(max_wait, base**i))

def call_model(cfg: Config, model: str, sys_msg: str, user_msg: str, max_tokens: int, temperature: float):
    def _do():
        return _chat(cfg.base_url, cfg.api_key, model,
                     [{"role":"system","content":sys_msg},{"role":"user","content":user_msg}],
                     max_tokens, temperature)
    text, usage = backoff_retry(_do)
    in_tok = usage.get("prompt_tokens", 0); out_tok = usage.get("completion_tokens", 0)
    return text.strip(), (in_tok + out_tok)

def exec_once(cfg: Config, prompt: str, task_text: str):
    msg = f"{prompt}\n\n### Task\n{task_text}\n\n### Answer"
    out, tokens = call_model(cfg, cfg.exec_model, SYS_EXEC, msg, cfg.max_tokens_exec, cfg.temperature_exec)
    return out, tokens

def judge_pair(cfg: Config, task_text: str, A: str, B: str):
    import random
    if random.random() < 0.5: A_, B_, swap = A, B, False
    else: A_, B_, swap = B, A, True
    jtxt, tokens = call_model(cfg, cfg.judge_model, SYS_JUDGE, judge_prompt(task_text, A_, B_)[1]["content"],
                              cfg.max_tokens_judge, cfg.temperature_judge)
    first_line = jtxt.splitlines()[0].strip().upper()
    label = "A" if ("A" in first_line) else ("B" if "B" in first_line else "A")
    if swap: label = "A" if label=="B" else "B"
    return label, "", tokens

def improve_prompts(cfg: Config, best_prompt: str, feedback: str, k: int):
    cands = []
    for _ in range(k):
        cand, _ = call_model(cfg, cfg.exec_model, OPT_SYS, opt_user(best_prompt, feedback),
                             max_tokens=512, temperature=0.7)
        cand = cand.strip()
        if cand and cand not in cands: cands.append(cand)
    return cands

def run(PROVIDER="mistral", TASK_FILE="tasks.json", INIT_PROMPT="prompt_seed.txt",
        ROUNDS=10, K=4, N=3, PATIENCE=2, OUT_DIR="runs/colab"):
    base = PROVIDERS[PROVIDER]
    key = os.getenv(base["key_env"], "")
    if not key: raise RuntimeError(f"Missing API key. Set {base['key_env']}")
    cfg = Config(provider=PROVIDER, api_key=key, base_url=base["base_url"],
                 exec_model="mistral-large-latest" if PROVIDER=="mistral" else "gpt-4o",
                 judge_model="mistral-small-latest" if PROVIDER=="mistral" else "gpt-4o-mini")
    cfg.rounds, cfg.k, cfg.n, cfg.patience, cfg.out_dir = ROUNDS, K, N, PATIENCE, OUT_DIR

    ensure_dir(cfg.out_dir); set_seed(cfg.seed)
    tasks = json.loads(pathlib.Path(TASK_FILE).read_text(encoding="utf-8"))
    init_prompt = pathlib.Path(INIT_PROMPT).read_text(encoding="utf-8")

    random.shuffle(tasks)
    train, valid = tasks[: max(3, cfg.n*2)], tasks[max(3, cfg.n*2): max(3, cfg.n*2)+max(2, cfg.n)]
    best_prompt = init_prompt
    history = []
    best_win = 0.0; no_gain = 0

    for r in range(1, cfg.rounds+1):
        pool = random.sample(train, k=min(cfg.n, len(train)))
        feedback = f"Prior best win rate={best_win:.2f}. Improve faithfulness, conciseness, coverage for tasks like: " + ", ".join(t["input"][:40]+"..." for t in pool)
        cands = [best_prompt] + improve_prompts(cfg, best_prompt, feedback, cfg.k)

        exec_tokens = 0; judge_tokens = 0
        results = {i: [] for i in range(len(cands))}
        for t in pool:
            outs = []
            for i, p in enumerate(cands):
                y, tk = exec_once(cfg, p, t["input"]); exec_tokens += tk; outs.append((i, y))
            base_idx = 0
            for i in range(1, len(outs)):
                (ia, ya), (ib, yb) = outs[base_idx], outs[i]
                lab, _, tkj = judge_pair(cfg, t["input"], ya, yb); judge_tokens += tkj
                win_idx = ia if lab=="A" else ib
                results[win_idx].append(1)

        winrates = {i: (sum(v)/max(1, len(v))) for i, v in results.items()}
        best_idx = max(winrates, key=lambda i: winrates[i])
        best_prompt = cands[best_idx]; best_round_win = winrates[best_idx]

        if best_round_win > best_win + 1e-6: best_win, no_gain = best_round_win, 0
        else: no_gain += 1

        history.append({
            "round": r, "best_winrate": best_round_win,
            "token_cost_exec": exec_tokens, "token_cost_judge": judge_tokens,
            "chosen_prompt": best_prompt
        })

        print(f"[Round {r}] best_winrate={best_round_win:.2f}  exec_tokens={exec_tokens} judge_tokens={judge_tokens}")
        if no_gain >= cfg.patience:
            print(f"Early stop at round {r} (no gain {no_gain} >= {cfg.patience})"); break

    # Save outputs
    import json
    pathlib.Path(cfg.out_dir, "trace.json").write_text(json.dumps({
        "config": asdict(cfg), "history": history, "best_prompt": best_prompt
    }, indent=2), encoding="utf-8")
    pathlib.Path(cfg.out_dir, "best_prompt.md").write_text(best_prompt, encoding="utf-8")
    pathlib.Path(cfg.out_dir, "run_report.md").write_text(
        "\n".join([
            "# Run Report", f"- Rounds executed: {len(history)}",
            f"- Best train winrate: {best_win:.2f}",
            f"- Token cost (sum): exec={sum(h['token_cost_exec'] for h in history)}, judge={sum(h['token_cost_judge'] for h in history)}",
            "- Output: best_prompt.md, trace.json"
        ]), encoding="utf-8"
    )
    print("Saved outputs to", cfg.out_dir)

if __name__ == "__main__":
    import argparse
    ap = argparse.ArgumentParser()
    ap.add_argument("--provider", default="mistral")
    ap.add_argument("--task_file", default="tasks.json")
    ap.add_argument("--init_prompt", default="prompt_seed.txt")
    ap.add_argument("--rounds", type=int, default=10)
    ap.add_argument("--k", type=int, default=4)
    ap.add_argument("--n", type=int, default=3)
    ap.add_argument("--patience", type=int, default=2)
    ap.add_argument("--out_dir", default="runs/colab")
    args = ap.parse_args()
    run(args.provider, args.task_file, args.init_prompt,
        args.rounds, args.k, args.n, args.patience, args.out_dir)
PYCODE
python - << 'PYTEST'
from pathlib import Path
print("Exists:", Path("spo_runner.py").exists())
PYTEST


## 5) Run optimization

In [ ]:

ROUNDS, K, N, PATIENCE = 6, 4, 3, 2
OUT_DIR = "runs/colab"
!python spo_runner.py --provider $PROVIDER --task_file tasks.json --init_prompt prompt_seed.txt   --rounds $ROUNDS --k $K --n $N --patience $PATIENCE --out_dir $OUT_DIR


## 6) Inspect outputs

In [ ]:

from pathlib import Path
print(Path("runs/colab/run_report.md").read_text())
print("\n--- best_prompt.md (first 600 chars) ---\n")
print(Path("runs/colab/best_prompt.md").read_text()[:600])
